<a href="https://colab.research.google.com/github/mohammadsjahanbakhsh/deep_learning_keras/blob/main/practise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers
from tensorflow.keras.models import Sequential
import tensorflow as tf

In [ ]:
!git clone https://github.com/mohammadsjahanbakhsh/deep_learning_keras

In [ ]:
!wget https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5

In [ ]:
n_class = 17
img_sizes = (240,240,3)
batch_size = 64
epochs = 30

In [ ]:
!unzip /content/deep_learning_keras/dataset/myflower2.zip
train_path = "/content/deep_learning_keras/dataset/17flowerclasses/train"
valid_path = "/content/deep_learning_keras/dataset/17flowerclasses/test"
my_flower = "/content/myflower2"

In [ ]:
# data augmentation and preprocessing with resnet50
trainGen = ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input,
    rotation_range=45,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    # rescale=1./255
)


train_gen_data = trainGen.flow_from_directory(
    train_path,
    target_size=img_sizes[:2],
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    seed=0
)

valid_Gen = ImageDataGenerator(
    preprocessing_function=keras.applications.resnet50.preprocess_input,
    # rescale=1./255
)

validation_gen_data = valid_Gen.flow_from_directory(
    valid_path,
    target_size=img_sizes[:2],
    batch_size=batch_size,
    class_mode="categorical",
    # subset="validation",
    seed=0 )


labels = dict((v,k) for k,v in train_gen_data.class_indices.items())
print(labels)

In [ ]:

weights_file_path = "/content/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
model = keras.models.Sequential([
    keras.applications.ResNet50(
        include_top = False,
        pooling = 'avg',
        weights = weights_file_path,
    ),

    layers.BatchNormalization(),
    layers.Dense(256,activation='relu'),
    layers.Dropout(.5),
    layers.Dense(128,activation='relu'),
    layers.Dropout(.5),
    layers.Dense(n_class,activation='softmax')
])
model.layers[0].trainable = False
print(model)


In [ ]:

model.summary()

In [ ]:

model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics=['accuracy']
      )

In [ ]:
history = model.fit(
            train_gen_data,
            validation_data=validation_gen_data,
            batch_size=batch_size,
            callbacks = [keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)] ,
            epochs=epochs,
            )

# vidualize loss & accuracy for Test & train

In [ ]:

fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(history.epoch , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(history.epoch, val_acc , 'ro-' , label = 'Validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")
ax[0].grid()

ax[1].plot(history.epoch, train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(history.epoch , val_loss , 'r-o' , label = 'Validation Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Training & Validation Loss")
ax[1].grid()
plt.show()

# پیشبینی و مصورسازی چند تصویر از خارج دیتاست

In [ ]:
predicted_class = train_gen_data.class_indices
train_gen_data.class_indices

In [ ]:
new_images=glob.glob(os.path.join(my_flower,"*/*.jpg"))
new_images

In [ ]:
def image_show(path):

  true_class=path.split("/")[3]

  img = tf.keras.preprocessing.image.load_img(path, target_size=(240, 240,3))
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array=keras.applications.resnet50.preprocess_input(img_array)
  img_array = tf.expand_dims(img_array, 0)
  predictions = int(np.argmax(model.predict(img_array),axis=1))
  plt.imshow(img)
  plt.title(f"true : {true_class}  , pred:  {predicted_class[predictions]}")
  plt.show()

image_show(new_images[0])

In [ ]:
image_show(new_images[1])

In [ ]:
image_show(new_images[2])

In [ ]:
image_show(new_images[3])

In [ ]:
image_show(new_images[4])

In [ ]:
image_show(new_images[5])